# Manim Differential Equation Visualization
## Visualizing dy/dx = -x/y with Direction Fields and Isoclines

This notebook demonstrates:
- Installing Manim in Google Colab
- Creating dynamic visualizations of differential equations
- Direction (slope) fields
- Isoclines (curves of constant slope)
- Solution curves

## Step 1: Install Manim and Dependencies

In [ ]:
# Install system dependencies
!sudo apt update
!sudo apt install libcairo2-dev ffmpeg \
    texlive texlive-latex-extra texlive-fonts-extra \
    texlive-latex-recommended texlive-science \
    tipa libpango1.0-dev -y

In [ ]:
# Install Manim
!pip install manim
!pip install IPython --upgrade

In [ ]:
# Uninstall problematic versions
!pip uninstall numpy scipy -y

# Install compatible versions
!pip install numpy==1.26.4
!pip install scipy==1.13.1

# Then install Manim
!pip install manim==0.18.1

## Step 2: Import Libraries

In [ ]:
from manim import *
import numpy as np

# Configure Manim for Colab
config.media_width = "75%"
config.verbosity = "WARNING"

## Step 3: Main Visualization - Direction Field with Isoclines

In [ ]:
%%manim -qm -v WARNING DifferentialEquationField

class DifferentialEquationField(Scene):
    def construct(self):
        # Title
        title = Text("dy/dx = -x/y", font_size=48)
        title.to_edge(UP)
        self.play(Write(title))
        self.wait()
        
        # Create axes
        axes = Axes(
            x_range=[-4, 4, 1],
            y_range=[-4, 4, 1],
            x_length=7,
            y_length=7,
            axis_config={"color": BLUE},
            tips=False,
        )
        
        axes_labels = axes.get_axis_labels(x_label="x", y_label="y")
        
        self.play(Create(axes), Write(axes_labels))
        self.wait()
        
        # Create vector field (direction field)
        def vector_field_func(pos):
            x, y = pos[0], pos[1]
            if abs(y) < 0.1:  # Avoid division by zero
                return np.array([0, 0, 0])
            dx = 1
            dy = -x / y
            # Normalize for consistent arrow lengths
            magnitude = np.sqrt(dx**2 + dy**2)
            if magnitude > 0:
                dx, dy = dx / magnitude * 0.3, dy / magnitude * 0.3
            return np.array([dx, dy, 0])
        
        vector_field = ArrowVectorField(
            vector_field_func,
            x_range=[-3.5, 3.5, 0.5],
            y_range=[-3.5, 3.5, 0.5],
            length_func=lambda norm: 0.3,
            colors=[YELLOW, ORANGE, RED]
        )
        
        field_label = Text("Direction Field", font_size=32, color=YELLOW)
        field_label.next_to(title, DOWN)
        
        self.play(Write(field_label))
        self.play(Create(vector_field), run_time=3)
        self.wait(2)
        
        # Draw isoclines
        isocline_label = Text("Isoclines (dy/dx = k)", font_size=28, color=GREEN)
        isocline_label.to_edge(LEFT).shift(UP * 2)
        self.play(Write(isocline_label))
        
        # Isoclines: dy/dx = k means -x/y = k, so y = -x/k (straight lines through origin)
        isocline_values = [-2, -1, -0.5, 0.5, 1, 2]
        isoclines = VGroup()
        
        for k in isocline_values:
            if k != 0:
                # y = -x/k
                isocline = axes.plot(
                    lambda x, k=k: -x / k,
                    x_range=[-3.5, 3.5],
                    color=GREEN,
                    stroke_width=2,
                    stroke_opacity=0.4
                )
                isoclines.add(isocline)
                
                # Label
                label_text = MathTex(f"k={k}", font_size=20, color=GREEN)
                if k > 0:
                    label_pos = axes.c2p(2, -2/k)
                else:
                    label_pos = axes.c2p(2, -2/k)
                label_text.move_to(label_pos)
                isoclines.add(label_text)
        
        self.play(Create(isoclines), run_time=2)
        self.wait(2)
        
        # Solution curves (circles centered at origin)
        solution_label = Text("Solution Curves: x² + y² = C", font_size=28, color=PINK)
        solution_label.to_edge(RIGHT).shift(UP * 2)
        self.play(Write(solution_label))
        
        # Draw several solution circles
        solution_curves = VGroup()
        radii = [0.5, 1, 1.5, 2, 2.5, 3]
        
        for r in radii:
            circle = Circle(
                radius=r * axes.x_axis.unit_size,
                color=PINK,
                stroke_width=3
            )
            circle.move_to(axes.c2p(0, 0))
            solution_curves.add(circle)
        
        self.play(Create(solution_curves), run_time=3)
        self.wait(2)
        
        # Animate a particle following one solution curve
        particle_label = Text("Particle Motion", font_size=28, color=BLUE)
        particle_label.to_edge(DOWN)
        self.play(Write(particle_label))
        
        dot = Dot(axes.c2p(2, 0), color=BLUE_B, radius=0.1)
        trace = TracedPath(dot.get_center, stroke_color=BLUE_B, stroke_width=4)
        self.add(trace)
        self.play(Create(dot))
        
        # Animate circular motion
        self.play(
            MoveAlongPath(dot, solution_curves[3], rate_func=linear),
            run_time=4
        )
        
        self.wait(2)
        
        # Final summary
        summary = VGroup(
            Text("Summary:", font_size=32),
            Text("• Direction field shows slope at each point", font_size=24),
            Text("• Isoclines connect points with same slope", font_size=24),
            Text("• Solutions are circles: x² + y² = C", font_size=24),
            Text("• Perpendicular to radial lines", font_size=24),
        ).arrange(DOWN, aligned_edge=LEFT, buff=0.3)
        summary.to_edge(LEFT).shift(DOWN * 1.5)
        
        self.play(
            FadeOut(particle_label),
            FadeOut(field_label),
            FadeOut(isocline_label),
            FadeOut(solution_label)
        )
        self.play(Write(summary))
        self.wait(3)

## Step 4: Alternative Visualization - Streamlines with Particles

In [ ]:
%%manim -qm -v WARNING InteractiveStreamplot

class InteractiveStreamplot(Scene):
    """Alternative visualization with streamlines and moving particles"""
    def construct(self):
        title = Text("Differential Equation: dy/dx = -x/y", font_size=40)
        title.to_edge(UP)
        self.play(Write(title))
        
        # Create axes
        axes = Axes(
            x_range=[-4, 4, 1],
            y_range=[-4, 4, 1],
            x_length=8,
            y_length=8,
            axis_config={"color": BLUE},
        )
        
        axes_labels = axes.get_axis_labels()
        self.play(Create(axes), Write(axes_labels))
        
        # Create streamlines (solution curves as circles)
        streamlines = VGroup()
        
        # Create circles at different radii with color gradient
        for r in np.linspace(0.5, 3.5, 15):
            circle = Circle(
                radius=r * axes.x_axis.unit_size,
                color=interpolate_color(BLUE, RED, r/3.5),
                stroke_width=2,
                stroke_opacity=0.7
            )
            circle.move_to(axes.c2p(0, 0))
            streamlines.add(circle)
        
        self.play(Create(streamlines), run_time=3)
        
        # Add multiple particles moving along the curves
        particles = VGroup()
        traces = VGroup()
        
        start_positions = [
            (3, 0), (2.5, 0), (2, 0), (1.5, 0),
            (0, 3), (0, 2.5), (0, 2), (0, 1.5)
        ]
        
        for x, y in start_positions:
            dot = Dot(axes.c2p(x, y), color=YELLOW, radius=0.08)
            particles.add(dot)
            trace = TracedPath(dot.get_center, stroke_color=YELLOW, stroke_width=2, stroke_opacity=0.5)
            traces.add(trace)
            self.add(trace)
        
        self.play(LaggedStart(*[Create(dot) for dot in particles], lag_ratio=0.1))
        
        # Animate all particles simultaneously
        animations = []
        for i, dot in enumerate(particles):
            x, y = start_positions[i]
            radius = np.sqrt(x**2 + y**2)
            circle_path = Circle(radius=radius * axes.x_axis.unit_size)
            circle_path.move_to(axes.c2p(0, 0))
            animations.append(MoveAlongPath(dot, circle_path, rate_func=linear))
        
        self.play(*animations, run_time=6)
        self.wait(2)

## Mathematical Explanation

### The Differential Equation: dy/dx = -x/y

**Solving the equation:**
- Separate variables: y dy = -x dx
- Integrate both sides: ∫y dy = ∫-x dx
- Result: y²/2 = -x²/2 + C
- Simplified: **x² + y² = C** (circles centered at origin)

**Key Concepts:**

1. **Direction Field (Slope Field):** Shows the slope dy/dx at each point (x,y). Each small arrow indicates the direction a solution would take at that point.

2. **Isoclines:** Curves where the slope dy/dx has the same constant value k.
   - For dy/dx = -x/y = k, we get y = -x/k
   - These are straight lines through the origin
   - Different k values give different isoclines

3. **Solution Curves:** The actual solutions to the differential equation
   - In this case: circles x² + y² = C
   - Each initial condition gives a different circle
   - Solutions are orthogonal (perpendicular) to radial lines from the origin

4. **Physical Interpretation:** This equation describes:
   - Circular motion
   - Level curves of the function f(x,y) = x² + y²
   - Flow perpendicular to the gradient

### Visualization Features:
- **Yellow/Orange/Red arrows:** Direction field showing slope at each point
- **Green lines:** Isoclines (constant slope lines)
- **Pink circles:** Solution curves
- **Blue dot with trace:** Example particle following a solution

The animation shows how solutions flow tangent to the direction field and perpendicular to the isoclines!

---

**Note:** Run each cell sequentially. The visualization will be displayed after the code cell completes.

To save the video, you can download it from the Colab file browser after rendering.